In [1]:


import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)

22/10/16 18:36:48 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
22/10/16 18:36:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-419243b9-6285-4430-8c9d-41ce791ebc72;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.1.0 in central
	found com.typesafe#config;1.3.0 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google

Spark NLP version 3.1.0
Apache Spark version: 3.0.2


In [2]:
from sparknlp.training import CoNLL
training_data = CoNLL().readDataset(spark, './eng_custom.train')
training_data.show()
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    text|            document|            sentence|               token|                 pos|               label|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    make|[[document, 0, 3,...|[[document, 0, 3,...|[[token, 0, 3, ma...|[[pos, 0, 3, 0, [...|[[named_entity, 0...|
|     can|[[document, 0, 2,...|[[document, 0, 2,...|[[token, 0, 2, ca...|[[pos, 0, 2, 0, [...|[[named_entity, 0...|
|      do|[[document, 0, 1,...|[[document, 0, 1,...|[[token, 0, 1, do...|[[pos, 0, 1, 0, [...|[[named_entity, 0...|
|    read|[[document, 0, 3,...|[[document, 0, 3,...|[[token, 0, 3, re...|[[pos, 0, 3, 0, [...|[[named_entity, 0...|
|   write|[[document, 0, 4,...|[[document, 0, 4,...|[[token, 0, 4, wr...|[[pos, 0, 4, 0, [...|[[named_entity, 0...|
|    talk|[[document, 0, 3,...|[[document, 0, 3,...|[[token, 0, 3, ta...

In [19]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en')\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)



nerTagger = NerDLApproach()\
    .setInputCols(["sentence", "token", "bert"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(100)\
    .setLr(0.003)\
    .setBatchSize(32)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')\
    .setTestDataset("test_withEmbeds.parquet")


test_data = CoNLL().readDataset(spark, './eng.testa')
test_data = bert.transform(test_data)

import os
 
# Directory name
directory = "test_withEmbeds.parquet"
 
# Parent Directory
parent = "./"
 
# Path
path = os.path.join(parent, directory)
 
# Remove the Directory
# "GeeksforGeeks"
try:
    os.rmdir(path)
    print("Directory '% s' has been removed successfully" % directory)
except OSError as error:
    print(error)
    print("Directory '% s' can not be removed" % directory)


test_data.write.parquet("test_withEmbeds.parquet")


ner_pipeline = Pipeline(stages = [bert, nerTagger])
ner_model = ner_pipeline.fit(training_data)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


22/10/16 18:45:47 WARN TaskSetManager: Stage 18 contains a task of very large size (1011 KiB). The maximum recommended task size is 1000 KiB.
2022-10-16 18:47:13.168855: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlic

Training started - total epochs: 100 - lr: 0.003 - batch size: 32 - labels: 7 - chars: 22 - training examples: 134
Epoch 1/100 started, lr: 0.003, dataset size: 134
Epoch 1/100 - 0.60s - loss: 35.239258 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.15s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 10	 42	 0	 0.1923077	 1.0	 0.32258064
Act	 0	 0	 16	 0.0	 0.0	 0.0
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 13	 5	 19	 0.7222222	 0.40625	 0.52
Obj	 3	 2	 12	 0.6	 0.2	 0.3
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 26 fp: 49 fn: 49 labels: 6
Macro-average	 prec: 0.25242165, rec: 0.26770833, f1: 0.25984037
Micro-average	 prec: 0.34666666, rec: 0.34666666, f1: 0.34666666
Quality on test dataset: 
time to finish evaluation: 4.60s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 28997	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 8	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 22203	 0	

Quality on test dataset: 
time to finish evaluation: 3.61s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 1824	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 2685	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 46712	 0	 0.0	 0.0	 0.0
Obj	 0	 41	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 12
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 9/100 started, lr: 0.0028846154, dataset size: 134
Epoch 9/100 - 0.08s - loss: 3.9621892 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 8	 2	 2	 0.8	 0.8	 0.8000001
Act	 12	 0	 4	 1.0	 0.75	 0.85714287
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 28	 0	 4	 1.0	 0.875	 0.93333334
Obj	 15	 10	 0	 0.6	 1.0	 0.

Quality on test dataset: 
time to finish evaluation: 3.60s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 794	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 4290	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 45989	 0	 0.0	 0.0	 0.0
Obj	 0	 189	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 12
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 16/100 started, lr: 0.0027906976, dataset size: 134
Epoch 16/100 - 0.08s - loss: 3.0549805 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 9	 0	 1	 1.0	 0.9	 0.9473684
Act	 12	 3	 4	 0.8	 0.75	 0.7741936
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 30	 0	 2	 1.0	 0.9375	 0.9677419
Obj	 13	 8	 2	 0.61904764	 

...

Quality on test dataset: 
time to finish evaluation: 3.55s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 173	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 2256	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 48170	 0	 0.0	 0.0	 0.0
Obj	 0	 663	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 12
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 72/100 started, lr: 0.0022140222, dataset size: 134
Epoch 72/100 - 0.07s - loss: 0.1778906 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 9	 0	 1	 1.0	 0.9	 0.9473684
Act	 12	 3	 4	 0.8	 0.75	 0.7741936
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 30	 0	 2	 1.0	 0.9375	 0.9677419
Obj	 13	 8	 2	 0.61904764	 

Quality on test dataset: 
time to finish evaluation: 3.84s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 356	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 1880	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 48480	 0	 0.0	 0.0	 0.0
Obj	 0	 546	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 12
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 79/100 started, lr: 0.0021582735, dataset size: 134
Epoch 79/100 - 0.08s - loss: 0.118257314 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 8	 0	 2	 1.0	 0.8	 0.88888896
Act	 12	 4	 4	 0.75	 0.75	 0.75
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 30	 0	 2	 1.0	 0.9375	 0.9677419
Obj	 13	 8	 2	 0.61904764	 0

Quality on test dataset: 
time to finish evaluation: 3.59s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 268	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 2554	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 47751	 0	 0.0	 0.0	 0.0
Obj	 0	 689	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 12
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 86/100 started, lr: 0.0021052633, dataset size: 134
Epoch 86/100 - 0.08s - loss: 0.20017864 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 8	 0	 2	 1.0	 0.8	 0.88888896
Act	 12	 4	 4	 0.75	 0.75	 0.75
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 30	 0	 2	 1.0	 0.9375	 0.9677419
Obj	 13	 7	 2	 0.65	 0.866666

Quality on test dataset: 
time to finish evaluation: 3.62s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 227	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 3308	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 46952	 0	 0.0	 0.0	 0.0
Obj	 0	 775	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 12
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 93/100 started, lr: 0.0020547945, dataset size: 134
Epoch 93/100 - 0.07s - loss: 0.044487905 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 9	 0	 1	 1.0	 0.9	 0.9473684
Act	 11	 3	 5	 0.78571427	 0.6875	 0.7333333
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 30	 1	 2	 0.9677419	 0.9375	 0.9523809
Obj	 13	 8

Quality on test dataset: 
time to finish evaluation: 3.60s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 128	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 3786	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 45916	 0	 0.0	 0.0	 0.0
Obj	 0	 1432	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 12
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 100/100 started, lr: 0.002006689, dataset size: 134
Epoch 100/100 - 0.08s - loss: 0.12651564 - batches: 5
Quality on validation dataset (20.0%), validation examples = 26
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 9	 0	 1	 1.0	 0.9	 0.9473684
Act	 11	 3	 5	 0.78571427	 0.6875	 0.7333333
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 30	 1	 2	 0.9677419	 0.9375	 0.9523809
Obj	 13	 

In [20]:
ner_model.stages[1].write().overwrite().save('NER_bert_20200221')

2022-10-16 18:54:39.308709: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [21]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

bert = BertEmbeddings.pretrained('bert_base_cased', 'en')\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)

loaded_ner_model = NerDLModel.load("NER_bert_20200221")\
   .setInputCols(["sentence", "token", "bert"])\
   .setOutputCol("ner")


ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_span')

custom_ner_pipeline = Pipeline(stages=[
      document_assembler, 
      sentence,
      tokenizer,
      bert,
      loaded_ner_model,
      ner_converter
 ])



bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


2022-10-16 18:54:47.887605: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [37]:
text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = custom_ner_pipeline.fit(prediction_data)
preds = prediction_model.transform(prediction_data)
preds.show()


+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               text|            document|            sentence|               token|                bert|                 ner|            ner_span|
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|navigate to kitchen|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 7, na...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 12, 18, ...|
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [38]:
import pyspark.sql.functions as F
#preds.select(F.explode(F.arrays_zip('ner_span.results','ner_span.metadata')).alias("entities"))\
#    .select(F.expr("entities['0']").alias("chunk")),(F.expr("entities['1']").alias("entity"))\
#    .show(truncate=False)

In [39]:
#preds.select('token.results','ner.results').show(truncate=False)

In [42]:
from sparknlp.base import LightPipeline
text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = custom_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

{'document': ['navigate to kitchen'], 'ner_span': ['kitchen'], 'token': ['navigate', 'to', 'kitchen'], 'ner': ['0', '0', 'Loc'], 'bert': ['navigate', 'to', 'kitchen'], 'sentence': ['navigate to kitchen']}
['0', '0', 'Loc']


['0', '0', 'Loc']
